credit: https://medium.com/fintechexplained/automating-stock-investing-fundamental-analysis-with-python-f854781ee0b4

In [11]:
import pandas
import yahoo_fin.stock_info as si


In [12]:
class Company:
    def __init__(self, symbol):
        self.symbol = symbol
        self.fundamental_indicators = {}
    

In [13]:
def to_float(val):
    if val == 0:
        return float(0)

    val = str(val).upper()

    if '%' in val:
        return round(float(val[:-1]), 4)

    m = {'K': 3, 'M': 6, 'B': 9, 'T': 12}

    for key in m.keys():
        if key in val:
            multiplier = m.get(val[-1])
            return round(float(val[:-1]) * (10 ** multiplier), 4)
    return round(float(val), 4)


def get_statatistics(symbol):
    url = f"https://finance.yahoo.com/quote/{symbol}/key-statistics?p={symbol}"
    dataframes = pandas.read_html(url)
    return pandas.concat(dataframes[1:])

def get_data_item(result, dataframe, columns):
    for column_to_find, column_to_name in columns.items():
        try:
            result[column_to_name] = list((dataframe.loc[dataframe[0] == column_to_find].to_dict()[1]).values())[0]
        except Exception as ex:
            result[column_to_name] = 'NA'

def get_last_data_item(result, dataframe, columns):
    data = dataframe.iloc[:, :2]
    data.columns = ["Column", "Last"]

    for column_to_find, column_to_name in columns.items():
        try:
            val = data[data.Column.str.contains(column_to_find, case=False, regex=True)].iloc[0, 1]
            float_val = to_float(val)
            result[column_to_name] = float_val
        except Exception as ex:
            result[column_to_name] = "NA"

In [14]:
def get_fundamental_indicators_for_company(config, company):
    company.fundmantal_indicators = {}

    # Statistics Valuation
    keys = {
        'Market Cap (intraday) 5': 'MarketCap',
        'Price/Sales (ttm)': 'PS',
        'Trailing P/E': 'PE',
        'PEG Ratio (5 yr expected) 1': 'PEG',
        'Price/Book (mrq)': 'PB'
     }
    data = si.get_stats_valuation(company.symbol)
#     print(data)
    get_data_item(company.fundamental_indicators, data, keys)

    # Income statement and Balance sheet
    data = get_statatistics(company.symbol)

    get_data_item(company.fundamental_indicators, data,
              {
                  'Profit Margin': 'ProfitMargin',
                  'Operating Margin (ttm)': 'OperMargin',
                  'Current Ratio (mrq)': 'CurrentRatio',
                  'Payout Ratio 4': 'DivPayoutRatio'
              })

    get_last_data_item(company.fundamental_indicators, data,
           {
               'Return on assets': 'ROA',
               'Return on equity': 'ROE',
               'Total cash per share': 'Cash/Share',
               'Book value per share': 'Book/Share',
               'Total debt/equity': 'Debt/Equity'
           })

In [15]:
config = {}
company = Company('ILMN')

get_fundamental_indicators_for_company(config, company)
print(company.fundamental_indicators)

{'MarketCap': '67.63B', 'PS': '19.47', 'PE': '108.23', 'PEG': '3.48', 'PB': '13.74', 'ProfitMargin': '18.14%', 'OperMargin': '16.04%', 'CurrentRatio': '4.45', 'DivPayoutRatio': '0.00%', 'ROA': 4.35, 'ROE': 13.18, 'Cash/Share': 31.71, 'Book/Share': 33.71, 'Debt/Equity': 58.45}


In [10]:
config = {}
company = Company('TMO')

get_fundamental_indicators_for_company(config, company)
print(company.fundamental_indicators)

NameError: name 'si' is not defined